# Clustering Crypto

In [86]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [87]:
# Load the crypto_data.csv dataset.
file_path = "./Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path,index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [88]:
# Keep all the cryptocurrencies that are being traded.
crypto_df=crypto_df[crypto_df["IsTrading"]== True]
crypto_df.shape

(1144, 6)

In [89]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df=crypto_df[crypto_df.Algorithm.notnull()]
crypto_df.shape
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [90]:
# Remove the "IsTrading" column. 
crypto_df=crypto_df.drop(['IsTrading'],axis=1)
crypto_df.shape

(1144, 5)

In [91]:
for column in crypto_df.columns:
    print(f"Column{column} has {crypto_df[column].isnull().sum()} null values")

ColumnCoinName has 0 null values
ColumnAlgorithm has 0 null values
ColumnProofType has 0 null values
ColumnTotalCoinsMined has 459 null values
ColumnTotalCoinSupply has 0 null values


In [92]:
# Remove rows that have at least 1 null value.
crypto_df=crypto_df.dropna()
crypto_df.shape

(685, 5)

In [93]:
crypto_df[crypto_df['TotalCoinsMined']<0.0]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
FIII,Fiii,SHA3-256,DPoC,-5.917978e+09,5000000000


In [94]:
crypto_df[crypto_df['TotalCoinsMined']==0.0]

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
808,808,SHA-256,PoW/PoS,0.0,0
XBS,Bitstake,X11,PoW/PoS,0.0,1300000
ACOIN,ACoin,SHA-256,PoW,0.0,1600000
AERO,Aero Coin,X13,PoS,0.0,7000000
APEX,ApexCoin,X13,PoW/PoS,0.0,6000000
...,...,...,...,...,...
QCN,Quazar Coin,CryptoNight,PoW,0.0,18446744
PKB,ParkByte,SHA-256,PoW/PoS,0.0,25000000
DOT,Dotcoin,Scrypt,PoW,0.0,890000000
THC,The Hempcoin,Scrypt,PoW/PoS,0.0,300000000


In [95]:
crypto_df[crypto_df['TotalCoinsMined']<=0.0]['CoinName'].count()
#685-153 = 532 (desired # for challenge)


153

In [96]:
# Keep the rows where coins are mined.
crypto_df=crypto_df[crypto_df['TotalCoinsMined']>0.0]
crypto_df.shape

(532, 5)

In [97]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_name_df = crypto_df[['CoinName']].copy()
crypto_name_df.shape

(532, 1)

In [98]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df=crypto_df.drop(['CoinName'],axis=1)

print(crypto_df.shape)
crypto_df

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [99]:
# Use get_dummies() to create variables for text features.
X=pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType'])
print(X.shape)
X

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
X_scaled[:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [101]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model 

pca = PCA(n_components=3,random_state=1)
X_pca = pca.fit_transform(X_scaled)

In [102]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=X_pca, columns =["PC 1","PC 2","PC 3"],index=crypto_df.index)

print(pcs_df.shape)
pcs_df

(532, 3)


,PC 1,PC 2,PC 3
42,-0.334216,1.059431,-0.611129
404,-0.317579,1.060258,-0.611764
1337,2.299018,1.740704,-0.746129
BTC,-0.141841,-1.332626,0.202796
ETH,-0.149732,-1.990729,0.406060
...,...,...,...
ZEPH,2.494434,0.344954,-0.066430
GAP,-0.332266,1.059398,-0.611174
BDX,0.319606,-2.282878,0.474041
ZEN,-0.156201,-2.110309,0.399649


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [116]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)



In [117]:
# Create an elbow curve to find the best value for K using hvPlot
elbow_data ={"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [191]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

#labels
labels = model.labels_
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [192]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

clustered_df = pd.concat([crypto_df,pcs_df],axis=1)
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] =crypto_name_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# predictions.dtype

clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334216,1.059431,-0.611129,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317579,1.060258,-0.611764,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.299018,1.740704,-0.746129,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.141841,-1.332626,0.202796,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.149732,-1.990729,0.406060,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.164912,-1.058848,-0.008733,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.398149,1.182077,-0.499508,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.155836,-2.253483,0.502534,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.148178,-1.990762,0.406026,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.156200,-2.110309,0.399649,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [193]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",  
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="Class",
    symbol="Class",
    width=1000,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [194]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE CoinName Algorithm ProofType TotalCoinSupply TotalCoinsMined Class

clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined','Class'],
                          sortable=True, selectable=True)



:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [195]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df['TotalCoinsMined'].count()} tradable cryptocurrencies")

There are 532 tradable cryptocurrencies


In [196]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
min_max_scaler = MinMaxScaler()

scaled_data= min_max_scaler.fit_transform(clustered_df[['TotalCoinSupply','TotalCoinsMined']])
scaled_data

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [197]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(scaled_data,columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"]=clustered_df["CoinName"]

# # Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"]=clustered_df["Class"]
plot_df.head(10)



,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [203]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply",by="Class",hover_cols=["CoinName"])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)